## slackから会話履歴とメンバーリストを読み込み、.jsonファイルに書き出します

In [1]:
# slackデータを抽出するslackアプリのトークンと、読み出し対象のチャンネルIDを設定します
TOKEN = "SLACK_API_TOKEN"
CHANNEL = "SLACK_CHANNEL_CODE"

# NEPTUNEの「接続とセキュリティ」の書き込みエンドポイントを設定します
ENDPOINT = "NEPTUNE_ENDPOINT_WRITE"

In [2]:
# Neptuneクラスターにつき一回だけ実行すれば良いです
!pip install slack_sdk

     |████████████████████████████████| 260 kB 21.9 MB/s            


In [3]:
import json
from slack_sdk import WebClient

In [4]:
def import_slack_for_neptune(token_str, channel_id):
    client = WebClient(token=token_str)
    response = client.conversations_history(channel=channel_id)
    user = client.users_list()
    members = user["members"]
    
    with open('./slack_for_neptune.json','w') as outfile:
        json.dump(response['messages'], outfile, indent=4)   
        outfile.write('\n')
    with open('./slack_users.json','w') as outfile:
        json.dump(user['members'], outfile, indent=4) 
        outfile.write('\n')

In [5]:
# Jupyter Notebook内にjsonファイルが二つ出来ます（後段の関数との兼ね合い）
import_slack_for_neptune(TOKEN, CHANNEL)

In [6]:
# 先ほどのjsonのうちusersの方は、csvへ変更しておきます（後段の関数との兼ね合い）
import csv

def create_csv_slack_members(json_path):
    write_fp=csv.writer(open("userIDs.csv", "w"))
    write_fp.writerow(['ID','name','name_e'])
    with open(json_path) as f:
        msgs = json.load(f)
    for msg in msgs:
        if ("name" in msg.keys()) & ("is_restricted" in msg.keys()):
            if msg["is_restricted"]==False:
                write_fp.writerow([msg["id"],msg["profile"]["display_name"],msg["name"]])

In [7]:
create_csv_slack_members("slack_users.json")

In [8]:
def full_members(json_path):
    with open(json_path) as f:
        msgs = json.load(f)
    for msg in msgs:
        if "name" in msg.keys():
            print('{},{}'.format(msg["id"],msg["profile"]["display_name"]))

In [9]:
def slack_members(json_path):
    with open(json_path) as f:
        msgs = json.load(f)
    for msg in msgs:
        if ("name" in msg.keys()) & ("is_restricted" in msg.keys()):
            if (msg["is_restricted"]==False) & (msg["profile"]["display_name"]!=''):
                print('{},{},{}'.format(msg["id"],msg["profile"]["display_name"],msg["name"]))

In [ ]:
slack_members("slack_users.json")

# ここから、jsonデータを再読み込みしてDataFrameにいれます

In [11]:
import pandas as pd

In [12]:
def extract_node_users(msgs):
    usernames = set([])
    for msg in msgs:
        if "user" in msg.keys():
            usernames.add(msg["user"])
    return pd.DataFrame({"ID": list(usernames)})

def extract_node_posts(msgs):
    texts = []
    ts =[]

    for msg in msgs:
        if "text" in msg.keys() and "ts" in msg.keys():
            ts.append(msg["ts"])
            texts.append(msg["text"])

    return pd.DataFrame({"post_id": ts,  "text":list(texts)})

def extract_edge_reactions(msgs):
    from_users = []
    to_posts = []
    names =[]

    for msg in msgs:

        if "reactions" in msg.keys():
            for reaction in msg["reactions"]:
                for user in reaction["users"]:
                    from_users.append(user)
                    to_posts.append(msg["ts"])
                    names.append(reaction["name"])


    #return pd.DataFrame({"~id": assign_index(to_posts), "from": from_users, "to":to_posts,"stamp_name":names})
    return pd.DataFrame({"ID": from_users, "post_id":to_posts,"stamp_name":names})

def extract_edge_post_actions(msgs):
    from_users = []
    to_posts = []
    types = []
    subtypes=[]

    for msg in msgs:
        if "ts" in msg.keys() and "user" in msg.keys():
            from_users.append(msg["user"])
            to_posts.append(msg["ts"])
            types.append(msg.get("types", ""))
            subtypes.append(msg.get("subtypes", ""))

    #return pd.DataFrame({"~id": assign_index(to_posts), "from": from_users,"to":to_posts, "type": types,"subtype": subtypes})
    return pd.DataFrame({"ID": from_users,"post_id":to_posts})


def dump_csv(df_name,df,path="./"):
    df.to_csv(os.path.join(path,f"{df_name}.csv"),index=False)
def send_users_to_neptune(json_path):
    with open(json_path) as f:
        return extract_node_users(json.load(f))
def send_posts_to_neptune(json_path):
    with open(json_path) as f:
        return extract_node_posts(json.load(f))
def send_edges_to_neptune(json_path):
    with open(json_path) as f:
        return extract_edge_reactions(json.load(f))
def send_edgeposts_to_neptune(json_path):
    with open(json_path) as f:
        return extract_edge_post_actions(json.load(f))


In [13]:
vertex_posts=send_posts_to_neptune("slack_for_neptune.json")
edge_reactions=send_edges_to_neptune("slack_for_neptune.json")
edge_posts=send_edgeposts_to_neptune("slack_for_neptune.json")
users = pd.merge(send_users_to_neptune("slack_for_neptune.json"), pd.read_csv("userIDs.csv"), on=['ID'])


In [ ]:
users

## ここからNeptune

In [15]:
%status

{'status': 'healthy',
 'startTime': 'Mon Jan 17 04:25:09 UTC 2022',
 'dbEngineVersion': '1.0.5.1.R3',
 'role': 'writer',
 'gremlin': {'version': 'tinkerpop-3.4.11'},
 'sparql': {'version': 'sparql-1.1'},
 'labMode': {'ObjectIndex': 'disabled',
  'DFEQueryEngine': 'viaQueryHint',
  'ReadWriteConflictDetection': 'enabled'},
 'features': {'ResultCache': {'status': 'disabled'},
  'IAMAuthentication': 'disabled',
  'Streams': 'disabled',
  'AuditLog': 'disabled'},
 'settings': {'clusterQueryTimeoutInMs': '120000'}}

In [17]:
%%gremlin
g.addV('character').property(id,'c1').property('name', 'アンパンマン').property('side', 'good').toSet()

In [18]:
%%gremlin
g.E().limit(10)

In [19]:
from gremlin_python import statics
from gremlin_python.structure.graph import Graph
from gremlin_python.process.graph_traversal import __
from gremlin_python.process.strategies import *
from gremlin_python.driver.driver_remote_connection import DriverRemoteConnection


In [20]:
DB_ENDPOINT=ENDPOINT
DB_PORT=8182
remoteConn = DriverRemoteConnection(f'wss://{DB_ENDPOINT}:{DB_PORT}/gremlin','g')
graph = Graph()
g = graph.traversal().withRemote(remoteConn)

In [21]:
g.V().drop().iterate()
g.E().drop().iterate()

[['E'], ['drop'], ['none'], ['values', '_ipython_canary_method_should_not_exist_'], ['values', '_ipython_canary_method_should_not_exist_']]

In [22]:
# userの頂点追加
for idx,user in users.iterrows():

    g.addV("user").property("user_id",user.ID).property('name', user[1]).toSet()
#postの頂点追加
for idx,post in vertex_posts.iterrows():
    g.addV("post").property("post_id",post.post_id).property('text', post.text).toSet()


In [ ]:
g.V().valueMap().toList()

In [24]:
for idx , edge in edge_posts.iterrows():
    
    print(edge.ID, edge.post_id)
    user = g.V().has('user_id', edge.ID).toList()
    
    if len(user):
        user = user[0]
    
    post = g.V().has('post_id', edge.post_id).toList()[0]
    g.addE('post').from_(user).to(post).toSet()

U0208FDCKG8 1642394693.040700
U0208FDCKG8 1642392097.039700
U0208FDCKG8 1642389106.038400
U0208FDCKG8 1642387343.037600
UMX1CPD6Y 1642386968.036800
UMX1CPD6Y 1642386617.033900
UMX1CPD6Y 1642382967.028900
UMX1CPD6Y 1642123507.027200
USLACKBOT 1642114801.026600
U02D3LXJEGN 1641963962.006400
UMX1CPD6Y 1641961974.003200
U0179UVTBQU 1641896414.002600
U0179UVTBQU 1641891769.002500
U02P03KAEM9 1641889882.000800
USLACKBOT 1640905230.081800
U02P03KAEM9 1640743371.078700
U02P03KAEM9 1640687938.076500
U0179UVTBQU 1640685956.073100
U0179UVTBQU 1640682169.071600
U01705LTTSB 1640675465.070600
U012DMZSBJ7 1640591975.066000
U02P03KAEM9 1640583433.063800
U012DMZSBJ7 1640066105.060500
UMX1CPD6Y 1640065398.059900
U02D3LXJEGN 1640065278.058600
UMX1CPD6Y 1640064972.057100
ULHA84QT1 1640064441.055700
ULHA84QT1 1640064403.055100
UMX1CPD6Y 1640064207.054900
ULHA84QT1 1640063272.053800
USLACKBOT 1639695604.052000
U012DMZSBJ7 1639478955.046200
ULHA84QT1 1639470314.040500
USLACKBOT 1638486027.037700
U012DMZSBJ7 

In [31]:
for idx , edge in edge_reactions.iterrows():
    
    print(edge.ID, edge.post_id)
    user = g.V().hasLabel('user').has('user_id', edge.ID).toList()
    #userの検索に引っかからないのがある。要調査
    if len(user):
        user = user[0]
    #以下のpostは読み込めない。IT-Admin系？
    if edge.post_id in [
    "1640675465.070600",
    "1640583433.063800",
    "1638436038.032800",
    "1634863088.000900",
    "1630661837.016600",
    "1630056167.010400"
    ]:
        continue
    post = g.V().hasLabel('post').has('post_id', edge.post_id).toList()[0]

    g.addE('stamp').from_(user).to(post).property('name',edge.stamp_name).toSet()


UMX1CPD6Y 1642386968.036800
UFLUZ76JX 1642386617.033900
U02P03KAEM9 1642123507.027200
UFLUZ76JX 1642123507.027200
U0179UVTBQU 1642123507.027200
U012DMZSBJ7 1641963962.006400
U02D3LXJEGN 1641961974.003200
U02P03KAEM9 1641961974.003200
U012DMZSBJ7 1641896414.002600
UMX1CPD6Y 1641896414.002600
U01705LTTSB 1640687938.076500
U0179UVTBQU 1640687938.076500
UMX1CPD6Y 1640687938.076500
U0179UVTBQU 1640687938.076500
UMX1CPD6Y 1640687938.076500
UD14Q3P6W 1640675465.070600
U012DMZSBJ7 1640675465.070600
UMX1CPD6Y 1640675465.070600
UF2N9JF7H 1640675465.070600
U02P03KAEM9 1640591975.066000
ULHA84QT1 1640583433.063800
UMX1CPD6Y 1640583433.063800
U019DE4M562 1640583433.063800
ULHA84QT1 1640066105.060500
UMX1CPD6Y 1640066105.060500
U02D3LXJEGN 1640065398.059900
U02D3LXJEGN 1640065398.059900
U01705LTTSB 1640065398.059900
ULHA84QT1 1640064972.057100
UMX1CPD6Y 1640064441.055700
UMX1CPD6Y 1639478955.046200
U012DMZSBJ7 1639470314.040500
U0179UVTBQU 1639470314.040500
UMX1CPD6Y 1638440323.036100
UMX1CPD6Y 1638

In [32]:
%%gremlin
g.V().valueMap()

In [33]:
%%gremlin
g.V().hasLabel('user').has('name', 'アンパンマン')

In [34]:
%%gremlin
g.V().hasLabel('user').has('name', 'ばいきんまん').outE('post').inV().values('text')

In [35]:
%%gremlin
g.E().valueMap()

In [36]:
%%gremlin
g.E().hasLabel('stamp').has('name', 'eyes').inV()

In [37]:
my_node_labels = '{"user":"name"}'
my_edge_labels = '{"stamp":"name"}'

In [38]:
%%gremlin -p v,oute,inv 
g.V().hasLabel("user").outE().inV().path().by(elementMap())

In [39]:
%%gremlin -p v,oute,inv -d $my_node_labels -de $my_edge_labels
g.V().hasLabel("user").outE().inV().path().by(elementMap())